# SAR BM3D (Block Matrix 3 Dimension)

#### Calculate SAR BM3D based on implementation by Y. Mäkinen, L. Azzari, A. Foi, "Exact Transform-Domain Noise Variance for Collaborative Filtering of Stationary Correlated Noise", in Proc. 2019 IEEE Int. Conf. Image Process. (ICIP), pp. 185-189. (https://pypi.org/project/bm3d/ ) More information: http://www.ipol.im/pub/art/2012/l-bm3d/

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import bm3d
import glob
import os

In [ ]:
def bm3d_filter(input_file, output_file):
    print('Begin filtering file ' + input_file)
    
    # inputing the file
    sar_image=gdal.Open(input_file)
    sar_band=sar_image.GetRasterBand(1)
    sar_band.GetMetadata()
    img_array=sar_band.ReadAsArray()
    [cols, rows]=img_array.shape

    noise_std_dev = 0.1
    
    filtered_image = bm3d.bm3d(img_array, noise_std_dev)
    
    # writing output file
    driver = gdal.GetDriverByName("GTiff")
    output = driver.Create(output_file, rows, cols, 1, gdal.GDT_Float32)
    proj = output.SetGeoTransform(sar_image.GetGeoTransform())  
    output.SetProjection(sar_image.GetProjection())
    output.SetGCPs(sar_image.GetGCPs(), "4326")
    geoband = output.GetRasterBand(1)
    geoband.WriteArray(filtered_image)
    output.GetRasterBand(1).SetNoDataValue(-9999)
    output.FlushCache()
    output = None
    band=None
    print('Created file ' + output_file)
    
    print('End filtering file ' + input_file)    

In [ ]:
for input_file in glob.glob('/home/cristina/Seeps/Workflow/Filters/ROI/*/NRCS/*tif'):
    output_file = input_file.split(os.sep)
    output_file[-1] = output_file[-1][:-4] + '_BM3D_10.tif'
    output_file = output_file[:-2] + ['Filters'] + ['Bilateral'] + output_file[-2:]
    output_file = os.sep.join(output_file)
    bm3d_filter(input_file, output_file)